In [1]:
import aiohttp
import asyncio
import bs4
import datetime
from io import BytesIO
import pandas as pd
import PyPDF2
import requests
import time
from tqdm.contrib.telegram import tqdm
from transliterate import to_latin
import json
import os

In [2]:
TOKEN = os.getenv("TOKEN")
CHAT_ID = os.getenv("CHAT_ID")

In [3]:
NEWS_URL = "https://president.uz/uz/lists/news?menu_id=12&page={page}&per-page=15" #1250
NEWS_LAST = 300

MEETING_URL = "https://president.uz/uz/lists/category/47?menu_id=12&page={page}&per-page=15"
MEETING_LAST = 41

FLY_URL = "https://president.uz/uz/lists/category/8?menu_id=12&page={page}&per-page=15"
FLY_LAST = 39

DELEG_URL = "https://president.uz/uz/lists/category/9?menu_id=12&page={page}&per-page=15"
DELEG_LAST = 35

CONGR_URL = "https://president.uz/uz/lists/category/7?menu_id=12&page={page}&per-page=15"
CONGR_LAST = 31

RULES_URL = "https://president.uz/uz/site/document/2?menu_id=144&page={page}&per-page=15"
RULES_LAST = 31

RULES2_URL = "https://president.uz/uz/site/document/1?menu_id=144&page=10&per-page=15"
RULES2_LAST = 52

In [4]:
URLS = [[NEWS_URL, NEWS_LAST], [MEETING_URL, MEETING_LAST], [FLY_URL, FLY_LAST], 
        [DELEG_URL, DELEG_LAST], [CONGR_URL, CONGR_LAST], [RULES_URL, RULES_LAST], [RULES2_URL, RULES2_LAST]]

In [5]:
BASE_URL = "https://president.uz"

In [17]:
from bs4 import BeautifulSoup

url = 'https://president.uz/uz/lists/view/6479'
response = requests.get(url)
soup = BeautifulSoup(response.text, 'lxml')
content = soup.find('div', class_="status_box_second").find_all('p')

# print(content)

for i in content[1:]:
    print(i.text)

Ўзбекистон Республикаси Президенти Шавкат Мирзиёевнинг Арманистон Республикаси Бош вазири Никол Пашинян билан 12 июль куни бўлиб ўтган телефон орқали мулоқотида икки томонлама муносабатларни, энг аввало, савдо-иқтисодий ва маданий-гуманитар соҳаларда ривожлантириш масалалари кўриб чиқилди.
Арманистон ҳукумати раҳбари мамлакатимиз етакчисини Президент сайловидаги ишончли ғалабаси билан табриклаб, катта муваффақиятлар ва бахт-саодат тилади.
Суҳбатда амалий ҳамкорлик лойиҳаларини илгари суриш мақсадида Ҳукуматлараро комиссиянинг биринчи йиғилиши ҳамда қўшма ишбилармонлик тадбирларини ўтказиш муҳимлиги алоҳида таъкидланди.


In [6]:
links = []

In [7]:
async def main():
    async with aiohttp.ClientSession() as session:
        
        for url in URLS:
            URL = url[0]
            LAST = url[1]
            
            for page in tqdm(range(1, LAST+1), token=TOKEN, chat_id=CHAT_ID):
                try:
                    async with session.get(URL.format(page=page)) as resp:
                        if resp.status == 200:
                            data = await resp.text()
                            soup = bs4.BeautifulSoup(data)
                            selected_links = soup.find('div', class_='tab-content').find_all('div', class_='news_box')
                            links.extend([BASE_URL+link.find('a')['href'] for link in selected_links])
                        elif resp.status != 404:
                            await asyncio.sleep(60) # 1 minute
                except Exception as e:
                    print(e, "page_num", page)
await main()

  0%|          | 0/300 [00:00<?, ?it/s]

  0%|          | 0/41 [00:00<?, ?it/s]

  0%|          | 0/39 [00:00<?, ?it/s]

  0%|          | 0/35 [00:00<?, ?it/s]

  0%|          | 0/31 [00:00<?, ?it/s]

  0%|          | 0/31 [00:00<?, ?it/s]

  0%|          | 0/52 [00:00<?, ?it/s]

In [9]:
with open(r'./HelperFiles/presidentuz.txt', 'w') as fp:
    for item in links:
        # write each item on a new line
        fp.write("%s\n" % item)
    print('Done')

Done


In [8]:
len(links)

7904

In [31]:
headers={'Accept': 'text/html'}

In [35]:
with open("proxies.json", "r", encoding="utf-8") as f:
    proxies = json.load(f)

In [37]:
proxies_ip = [proxy['ip'] for proxy in proxies]

['208.113.220.98',
 '181.129.186.130',
 '203.143.38.249',
 '92.222.237.105',
 '138.97.0.66',
 '75.119.205.47',
 '81.12.124.38',
 '78.137.58.119',
 '213.149.103.132',
 '180.183.1.84',
 '203.176.141.166',
 '43.153.177.245',
 '103.199.97.33',
 '41.58.136.94',
 '31.206.38.49',
 '103.35.109.142',
 '189.201.191.19',
 '213.135.234.101',
 '103.36.102.182',
 '110.78.149.89',
 '38.127.179.194',
 '114.29.212.145',
 '202.144.157.1',
 '66.33.210.111',
 '104.36.7.57',
 '200.8.185.126',
 '211.144.20.13',
 '165.225.24.50',
 '201.121.249.246',
 '45.7.176.78',
 '201.65.121.66',
 '45.7.177.241',
 '116.199.170.17',
 '5.190.229.170',
 '170.150.159.18',
 '36.66.36.252',
 '81.68.156.53',
 '103.251.83.61',
 '188.120.248.106',
 '223.155.121.75',
 '155.133.83.109',
 '103.59.203.133',
 '43.134.230.127',
 '89.237.36.193',
 '212.34.239.253',
 '203.99.57.145',
 '91.187.120.12',
 '103.109.59.109',
 '202.44.228.181',
 '177.66.195.118',
 '1.0.136.60',
 '198.211.38.45',
 '185.205.249.24',
 '190.207.141.181',
 '185.162.

In [50]:
with open('proxies.txt') as f:
    lines = f.readlines()

In [56]:
proxies_ip = lines[210:]

In [57]:
results = []

async def main():
    async with aiohttp.ClientSession() as session:
        proxy = proxies_ip[0]

        max_v = 500
        
        
        for ind, link in enumerate(tqdm(links, token=TOKEN, chat_id=CHAT_ID)):
            try:
                proxy_auth = aiohttp.BasicAuth('user', 'pass')
                if ind > max_v:
                    proxy = proxies_ip[proxies_ip.index(proxy) + 1]
                    max_v += 500
                async with session.get(link, proxy=f"http://{proxy}", proxy_auth=proxy_auth) as resp:
                    if resp.status == 200:
                        data = await resp.text()
                        soup = bs4.BeautifulSoup(data)

                        content = soup.find('div', class_="status_box_second").find_all('p')
                        title = soup.find('div', class_="pre_full_name").text

                        content = "".join(i.text for i in content[1:])
                        results.append({
                            "title": title.strip(),
                            "content": content.strip()
                        })
                    elif resp.status != 404:
#                         await asyncio.sleep(60) # 1 minute
                        print("404", link)
            except Exception as e:
#                 print(title)
#                 print(content)
                print(e, link)
    
await main()

  0%|          | 0/7904 [00:00<?, ?it/s]

500, message='Internal Privoxy Error', url=URL('http://139.59.59.122:8118') https://president.uz/uz/lists/view/6487
[Errno 104] Connection reset by peer https://president.uz/uz/lists/view/6486
500, message='Internal Privoxy Error', url=URL('http://139.59.59.122:8118') https://president.uz/uz/lists/view/6485


CancelledError: 

In [13]:
results = []

for ind, link in enumerate(tqdm(links, token=TOKEN, chat_id=CHAT_ID)):
    try:
        resp = requests.get(link)
        if resp.status_code == 200:
            data = resp.text
            soup = bs4.BeautifulSoup(data)

            content = soup.find('div', class_="status_box_second").find_all('p')
            title = soup.find('div', class_="pre_full_name").text

            content = "".join(i.text for i in content[1:])
            results.append({
                "title": title.strip(),
                "content": content.strip()
            })
        else:
    #                         await asyncio.sleep(60) # 1 minute
            print(resp.status_code, link)
    except Exception as e:
#                 print(title)
#                 print(content)
        print(e, link)
    

  0%|          | 0/7904 [00:00<?, ?it/s]

In [14]:
len(results)

7904

In [20]:
df = pd.DataFrame(results)

In [23]:
# df

,title,content
0,Ўзбекистон Президенти амалий ташриф билан Сауд...,Икки муқаддас масжид ходими – Саудия Арабистон...
1,Хорижий ҳамкорлардан самимий қутловлар,Ўзбекистон Республикаси Президенти Шавкат Мирз...
2,Ўзбекистон Республикасининг сайланган Президен...,"14 июль куни Тошкентда, Ўзбекистон Республикас..."
3,"Халқимизнинг юксак ишончини оқлаш, эришган юту...",Янги сайланган Ўзбекистон Республикаси Президе...
4,Ўзбекистон Президенти Туркия билан фаол парлам...,Ўзбекистон Республикаси Президенти Шавкат Мирз...
...,...,...
7899,Д.М.Кенжаевни Ўзбекистон Республикаси Президен...,Дилшод Муталибович Кенжаев Ўзбекистон Республи...
7900,Д.А.Хидоятовни Тошкент вилояти ҳокими вазифаси...,Ўзбекистон Республикаси Конституциясининг 93-м...
7901,Р.Қ.Холматовни Тошкент вилояти ҳокими лавозими...,Ўзбекистон Республикаси Конституциясининг 93-м...
7902,Д.Б.Кадировни Ўзбекистон Республикаси Президен...,Дониёр Бахтиёрович Кадиров Ўзбекистон Республи...


In [25]:
from transliterate import to_latin

df['title'] = df['title'].apply(lambda x: to_latin(x))

df['content'] = df['content'].apply(lambda x: to_latin(x))

In [27]:
# pd.DataFrame(results)

,title,content
0,Namangan viloyati IIB boshligʻi huzurida Chust...,"Unda Boshqarma boshligʻi oʻrinbosarlari, Boshq..."
1,Namangan viloyati IIB Majlislar zalida Migrats...,"Unda, viloyat IIB boshligʻi oʻrinbosarlari, so..."
2,Denov tumanida tashkilotlar oʻrtasida sportnin...,Joriy yilning 15-iyul kuni Denov tumanidagi ma...
3,Surxondaryo viloyati IIB Migratsiya va fuqarol...,Joriy yilning 15-iyul kuni Surxondaryo viloyat...
4,Qumqoʻrgʻon tumani IIBda yangi “Kutubxona” foy...,2023-yil 14-iyul kuni Qumqoʻrgʻon tumani Ichki...
...,...,...
1263,“14 yanvar` – Vatan himoyachilari kuni” bayram...,Barcha joyda bo'lgani kabi O'zbekiston Respubl...
1264,Guliston shahrida “Profilaktika kuni” munosaba...,Guliston shahar Qurilish kasb-hunar kollejida ...
1265,Bir guruh ichki ishlar organlari xodimlari taq...,Ichki ishlar vazirligi tizimi xodimlarining ke...
1266,"""Vatanparvarlik oyligi"" davom etmoqda",IIV Akademiyasi Axborot xizmatining xabar beri...


In [26]:
df.to_pickle("MainFiles/presidentuz.pickle")